In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets.mnist import load_data

In [3]:
# load the images into memory
(trainX, trainy), (testX, testy) = load_data() 

# summarize the shape of the dataset 
print('Train', trainX.shape, trainy.shape) 
print('Test', testX.shape, testy.shape)

11490434/11490434 [==============================] - 0s 0us/step
Train (60000, 28, 28) (60000,)
Test (10000, 28, 28) (10000,)
